In [2]:
import datasets
import torch
import transformers
from tqdm.autonotebook import tqdm

In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("speakleash/Bielik-1.5B-v3.0-Instruct")
model = AutoModelForCausalLM.from_pretrained("speakleash/Bielik-1.5B-v3.0-Instruct").to(DEVICE)

In [ ]:
dataset = datasets.load_dataset("Igorrr0/polish-qa-general")


def to_chat_format(sample):
    system_prompt = {"role": "system", "content": "Odpowiadaj krótko i konwersacyjnie :)"}
    user_msg = {"role": "user", "content": sample["instruction"]}
    assistant_msg = {"role": "assistant", "content": sample["output"]}
    return {"user": [system_prompt, user_msg], "output": [system_prompt, user_msg, assistant_msg]}

dataset = dataset.map(to_chat_format).remove_columns(["instruction", "input"])

dataset["train"][0]

Map:   0%|          | 0/368 [00:00<?, ? examples/s]

{'output': [{'content': 'Odpowiadaj krótko i konwersacje', 'role': 'system'},
  {'content': 'Jak mogę pomóc?', 'role': 'user'},
  {'content': 'Czym mogę służyć?', 'role': 'assistant'}],
 'user': [{'content': 'Odpowiadaj krótko i konwersacje', 'role': 'system'},
  {'content': 'Jak mogę pomóc?', 'role': 'user'}]}

In [ ]:
import re

import pandas as pd


generated_dataset = []


def custom_chat_template(messages):
    return f"""<|im_start|> system
    {messages[0]["content"]}<|im_end|> 
    <|im_start|> user
    {messages[1]["content"]}<|im_end|> 
    <|im_start|> assistant
    """


def extract_assistant_message(text):
    pattern = r"<\|im_start\|\>\s*assistant\s*(.*?)<\|im_end\|>"
    m = re.search(pattern, text, re.DOTALL)
    if m:
        msg = m.group(1).strip()
        return msg
    return ""


for sample in dataset["train"]:
    user_tokenized = tokenizer(custom_chat_template(sample["user"]), return_tensors="pt")

    model_inputs = user_tokenized.to(DEVICE)
    generated_ids = model.generate(**model_inputs, max_new_tokens=1024, do_sample=True)
    decoded = tokenizer.decode(generated_ids[0])
    print("\n==========")
    print(decoded)
    print("==========\n")

    label = sample["user"] + [{"role": "assistant", "content": extract_assistant_message(decoded)}]
    generated_dataset += [{"messages": label}]

df = pd.DataFrame(generated_dataset)
df.to_csv('./cached_ds.csv', index=False)


<s><|im_start|>  system
    Odpowiadaj krótko i konwersacje<|im_end|>  
    <|im_start|>  user
    Jak mogę pomóc?<|im_end|>  
    <|im_start|>  assistant
    1. Czy mogę Ci podać aktualne wiadomości?
   - Oczywiście.  
   2. Szukasz instrukcji dotyczącej użycia konkretnego programu?  
   - Zapraszam do zadawania pytań.  
   3. Pomogę Ci w organizacji spotkań lub podróży.  
   - Proszę podaj szczegóły.  
   4. Potrzebujesz wsparcia w nauce lub zadaniu domowym.  
   - Służę pomocą.  
   5. Czy chcesz zaplanować coś ważnego (np. zakupy, rezerwację)?
   - Proszę określ cel.  
   
   A jeśli jesteś tu pierwszy raz, polecam zapoznaj się z funkcją "ustawienia systemu".<|im_end|>


<s><|im_start|>  system
    Odpowiadaj krótko i konwersacje<|im_end|>  
    <|im_start|>  user
    Cześć!<|im_end|>  
    <|im_start|>  assistant
      Cześć! Nie ma sprawy, jak mogę pomóc dziś?<|im_end|>


<s><|im_start|>  system
    Odpowiadaj krótko i konwersacje<|im_end|>  
    <|im_start|>  user
    Dzień dob

KeyboardInterrupt: 